# 1. Clone darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

# Cuda get version

In [ ]:
! /usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# 2. Update makefile

In [ ]:
%cd darknet

In [5]:
# change makefile to have GPU and OPENCV enabled
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# 3. Make

In [ ]:
!make

# 4. Download model

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

# 5. Download config model

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Create simple script for show Result detect

In [9]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file
def download(path):
  from google.colab import files
  files.download(path)

# Run detect objects

In [ ]:
# run darknet detection on test images
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

# Show result after detect

In [ ]:
# show image using our helper function
imShow('predictions.jpg')

# Detect object on local images

In [ ]:
# run darknet detection on test images
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights ../data/street.jpg

# Create data folder

In [12]:
!mkdir data/custom

In [16]:
!mv data/test.zip data/custom/

# Train custom model

In [ ]:
!cd data/custom && unzip test.zip

In [ ]:
!./darknet detector train data/custom/mask.data data/custom/mask.cfg yolov4.conv.137 -dont_show -map

In [ ]:
!cat data/custom/mask.data

# Detect object use custom model

In [ ]:
!./darknet detector test data/custom/mask.data data/custom/mask_test.cfg backup/mask_last.weights data/custom/obj/0.jpg -thresh 0.3

# Check average precision

In [ ]:
!./darknet detector map data/custom/mask.data data/custom/mask.cfg backup/mask_last.weights -points 0